In [2]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import random
import math 
import copy
import random
import argparse
import torch.optim as optim
import torch.nn as nn
import numpy as np
from scipy.optimize import minimize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import numpy as np

cuda:0


In [66]:
import torch
import torch.nn as nn

# 定义一个简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu2 = nn.ReLU()

        # 使用Xavier初始化来初始化权重
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)

        # 初始化偏置为零
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.constant_(self.fc2.bias, 0)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        return out
        
# 定义输入、隐藏层和输出的维度
input_size = 10
hidden_size = 20
output_size = 10

# 创建一个SimpleNN模型



In [39]:
class customloss(nn.Module):
    def __init__(self, theta):
        super(customloss, self).__init__()
        self.theta = theta
    def forward(self, output_list, y_list):
        theta = torch.tensor(self.theta, dtype= torch.float32).to(device) 
        loss = 0
        index = 0
        for output in output_list:  
            y = torch.tensor(y_list[index]).to(device) 
            v = torch.mm(output, theta.view(-1,1)) 
            prob = torch.exp(v) / (torch.sum(torch.exp(v)))  
            loss += torch.sum(-y * torch.log(prob) - (1-y) * torch.log(1-prob)) 
            index += 1  
        return loss / (len(y_list)/100)

In [26]:
CONTEXT_ARRAY = np.load('features1.npy')
theta = np.load('theta.npy')
PURCHASE_LIST = np.load('purchase.npy')

In [30]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3090'

In [54]:
model = SimpleNN(input_size, hidden_size, output_size).to(device)
print(model)

SimpleNN(
  (fc1): Linear(in_features=10, out_features=20, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=20, out_features=10, bias=True)
)


In [89]:
feature_list = []
purchase_list = []

model = SimpleNN(input_size, hidden_size, output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)
for t in range(0,1000):
    context = CONTEXT_ARRAY[t]

    purchase_list.append(PURCHASE_LIST[t])

    feature_list.append(np.array(context))

    if t%100 == 99:
        loss_function = customloss(theta)
        epochs = 5
        for epoch in range(epochs):
            
            output_list = [model(torch.tensor(a,dtype = torch.float32).to(device)) for a in feature_list]
            loss = loss_function(output_list, purchase_list)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
            print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 2419.822021484375
epoch 1, loss 2416.7216796875
epoch 2, loss 2413.093505859375
epoch 3, loss 2412.091796875
epoch 4, loss 2411.818359375
epoch 0, loss 2411.72900390625
epoch 1, loss 2411.63037109375
epoch 2, loss 2411.55126953125
epoch 3, loss 2411.481689453125
epoch 4, loss 2411.43017578125
epoch 0, loss 2411.404541015625
epoch 1, loss 2411.38916015625
epoch 2, loss 2411.37939453125
epoch 3, loss 2411.372802734375
epoch 4, loss 2411.369140625
epoch 0, loss 2411.3623046875
epoch 1, loss 2411.360107421875
epoch 2, loss 2411.35791015625
epoch 3, loss 2411.3564453125
epoch 4, loss 2411.354736328125
epoch 0, loss 2411.3505859375
epoch 1, loss 2411.349365234375
epoch 2, loss 2411.34912109375
epoch 3, loss 2411.34814453125
epoch 4, loss 2411.347412109375
epoch 0, loss 2411.34521484375
epoch 1, loss 2411.344482421875
epoch 2, loss 2411.34423828125
epoch 3, loss 2411.343994140625
epoch 4, loss 2411.34326171875
epoch 0, loss 2411.3408203125
epoch 1, loss 2411.340576171875
epoch 2

In [84]:
model = SimpleNN(input_size, hidden_size, output_size).to(device)

In [88]:
problist  = np.load('prob_list.npy')
def hhh(i):
    ff = model(torch.tensor(CONTEXT_ARRAY[i],dtype = torch.float32).to(device))
    ff = ff.cpu().detach().numpy()
    u = (ff @ theta.T ).reshape(1,-1)
    prob = np.exp(u) / (np.sum(np.exp(u)))
    
    print(problist[i])
    print(prob)

hhh(100)   

[0.07958204 0.22451541 0.24934586 0.08885355 0.05108825 0.08780105
 0.09688146 0.12193237]
[[0.12208319 0.12550613 0.12462931 0.12528159 0.12241695 0.12624263
  0.12695148 0.12688872]]


In [63]:
theta2 = torch.tensor(theta, dtype= torch.float32).to(device) 

for output in output_list:  
    y = torch.tensor(y_list[index]).to(device) 
    v = torch.mm(output, theta2.view(-1,1)) 
    prob = torch.exp(v) / (torch.sum(torch.exp(v)))  

prob

NameError: name 'y_list' is not defined